In [4]:
import sqlite3

In [5]:
DATABASE_PATH = "data.db"

In [7]:
con = sqlite3.connect(DATABASE_PATH)

In [8]:
cur = con.cursor()

backed up profiles old:

In [6]:
cur.execute("ALTER TABLE profiles RENAME TO profiles_old")


In [9]:
cur.execute("CREATE TABLE profiles("
            "value TEXT PRIMARY KEY,"
            "datasetName TEXT,"
            "datasetSize INTEGER,"
            "description TEXT,"
            "imageHeight INTEGER,"
            "imageWidth INTEGER,"
            "randomOrientation TEXT,"
            "CVPercentage INTEGER,"
            "TestSetPercentage INTEGER)")

In [10]:
cur.execute("INSERT INTO profiles (value, datasetName, datasetSize, description, imageHeight, imageWidth, randomOrientation, CVPercentage, TestSetPercentage)"
"SELECT value, datasetName, datasetSize, description, imageHeight, imageWidth, randomOrientation, CVPercentage, TestSetPercentage"
"FROM profiles_old")

OperationalError: no such column: value

In [6]:
con.commit()
con.close()

In [17]:
cur.execute("PRAGMA table_info(profiles_old)")

In [18]:
print(con.cursor().fetchall())

[]


In [20]:
cur.execute("""
        INSERT INTO profiles (
            value, datasetName, datasetSize, description,
            imageHeight, imageWidth, randomOrientation, CVPercentage
        )
        SELECT 
            value, datasetName, datasetSize, description,
            imageHeight, imageWidth, randomOrientation, CVPercentage
        FROM profiles_old;
    """)

In [21]:
con.commit()

In [22]:
cur.execute("DROP TABLE profiles_old;")

In [23]:
con.commit()

In [24]:
cur.execute("""
        ALTER TABLE datasets ADD COLUMN split TEXT;

""")

In [26]:
con.commit()

In [49]:
cur.execute('DELETE FROM datasets WHERE value = ?', ("4e9ecbf2-15-02-2025-1407-49",))
con.commit()

In [42]:
cur.execute("INSERT INTO datasets ("
                "value, " 
                "datasetName, "
                "datasetSize, "
                "description, "
                "imageHeight, "
                "imageWidth, "
                "randomOrientation )"
                "VALUES (?, ?, ?, ?, ?, ?, ?) ON CONFLICT(value) DO UPDATE SET "
                "datasetName = excluded.datasetName, "
                "datasetSize = excluded.datasetSize, "
                "description = excluded.description, "
                "imageHeight = excluded.imageHeight, "
                "imageWidth = excluded.imageWidth, "
                "randomOrientation = excluded.randomOrientation",
                ("820990fd-21-05-2025-1631-11", 
                "DatasetProf2", 
                1000, 
                "Random orientation. Basic 500x500 dataset of 1000, will initially be used to test checkpoint loading. Test to check generator still works", 
                500, 
                500, 
                1))

In [45]:
con.commit()

## Creating tables for training runs and training data: 

In [19]:
cur.execute("""CREATE TABLE training_runs (
    id TEXT PRIMARY KEY,
    model_id TEXT,
    checkpoint TEXT,
    training_dataset TEXT,
    cv_dataset TEXT,
    test_dataset TEXT,
    optimiser TEXT,
    loss_function TEXT,
    learning_rate REAL,
    epochs INTEGER,
    session_id TEXT
);""")

In [20]:
con.commit()

In [11]:
cur.execute("""CREATE TABLE training_metrics (
    run_id TEXT,
    epoch INTEGER,
    batch INTEGER,
    loss REAL,
    FOREIGN KEY(run_id) REFERENCES training_runs(id)
);""")

In [12]:
con.commit()

In [17]:
cur.execute("""DROP TABLE training_runs""")

In [18]:
con.commit()